In [ ]:
import pandas as pd
from yahoofinancials import YahooFinancials

selected_date = "2024-04-21"
# selected_date = "2020-09-01"
date = pd.to_datetime(selected_date)
start_date = str((date - pd.Timedelta(days=13)).date())
end_date = str(date.date())
yahoo_financials = YahooFinancials("BTC-USD")
df = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
prices = df["BTC-USD"]["prices"]
prices

In [ ]:
# Preprocess for prediction
df = pd.DataFrame(prices)
df["date"] = pd.to_datetime(df["formatted_date"])
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df.dropna(inplace=True)
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df.rename(columns={"volume": "Volume"}, inplace=True)
for i in range(1, 8):
    df[f"lag_close_{i}"] = df["close"].shift(i)
df.dropna(inplace=True)
df

In [ ]:
from joblib import load

# Load trained model from a file
try:
    file_name = "trained_models/bitsmart_model_1715639499"
    models = load(file_name)
    model_open = models["open"]
    model_high = models["high"]
    model_low = models["low"]
    model_close = models["close"]
    model_avg = models["average"]
except Exception as e:
    print(e)

In [ ]:
def predict_next_7_days(selected_date, data):
    selected_date = pd.to_datetime(selected_date)
    prediction_results = []
    for i in range(7):
        base_date = selected_date - pd.Timedelta(days=i)
        record = data[data["date"] == base_date]
        feature_names = [
            "lag_close_1",
            "lag_close_2",
            "lag_close_3",
            "lag_close_4",
            "lag_close_5",
            "lag_close_6",
            "lag_close_7",
            "Volume",
        ]
        feature_vector = record[feature_names].values.reshape(1, -1)
        feature_vector_df = pd.DataFrame(feature_vector, columns=feature_names)
        predicted_open = model_open.predict(feature_vector_df)[0]
        predicted_high = model_high.predict(feature_vector_df)[0]
        predicted_low = model_low.predict(feature_vector_df)[0]
        predicted_close = model_close.predict(feature_vector_df)[0]
        predicted_average = model_avg.predict(feature_vector_df)[0]
        prediction_results.insert(0,
            {
                "date": (base_date + pd.Timedelta(days=7)).strftime("%Y-%m-%d"),
                "open": predicted_open,
                "high": predicted_high,
                "low": predicted_low,
                "close": predicted_close,
                "average": predicted_average,
            }
        )
    return prediction_results

In [ ]:
predictions = predict_next_7_days(selected_date, df)
predictions

In [ ]:
def swing_trading_strategy(data, investment_amount):
    if data.empty:
        raise ValueError("No data available")

    # Initialize trading state
    start_date = data["date"][0]
    initial_open_price = data["open"][0]
    bitcoins = investment_amount / initial_open_price
    cash = 0
    sell_executed = False
    load_executed = False
    sell_day = None
    load_day = None
    total_avg_closing_price = 0
    number_of_days = len(data.index)

    for i in range(number_of_days):
        base_date = start_date + pd.Timedelta(days=i)
        price = data[data["date"] == base_date]
        price_high = price["open"].values[0]
        price_low = price["low"].values[0]
        price_avg = price["average"].values[0]

        # Accumulate average closing prices
        total_avg_closing_price += price_avg

        # Decide when to sell: if predicted high price is 5% higher than the open price
        if not sell_executed and price_high > initial_open_price * 1.05:
            sell_executed = True
            sell_day = base_date
            cash = bitcoins * price_high
            bitcoins = 0

        # Decide when to load: after selling, if the predicted low price is 5% lower
        if sell_executed and not load_executed and price_low < price_high * 0.95:
            # Ensure not loading on the sell day
            if base_date != sell_day:
                load_executed = True
                load_day = base_date
                bitcoins = cash / price_low
                cash = 0

    # Calculate average closing price over the prediction period
    average_closing_price = total_avg_closing_price / number_of_days

    # Final valuation at the end of the prediction period or the last available day
    final_day = start_date + pd.Timedelta(days=number_of_days - 1)
    final_close_price = data.loc[data["date"] == final_day, "close"].values[0]
    final_cash = cash if cash > 0 else bitcoins * final_close_price

    # Calculate profit/loss
    profit_loss = final_cash - investment_amount

    return {
        "profit_loss": profit_loss,
        "total_investment_amount": investment_amount,
        "total_exit_amount": final_cash,
        "average_closing_price": average_closing_price,
        "sell_day": str(sell_day.date()) if sell_day else "NA",
        "load_day": str(load_day.date()) if load_day else "NA",
        "final_bitcoins": bitcoins if cash == 0 else 0,
    }

In [ ]:
def swing_trading_strategy(data, investment_amount):
    if data.empty:
        raise ValueError("No data available")

    # Initialize trading state
    number_of_days = len(data.index)
    initial_open_price = data["open"][0]
    investment_amount = float(investment_amount)
    initial_bitcoins = investment_amount / initial_open_price
    last_closing_amount = data["close"][number_of_days - 1]

    # Hold
    strategies = [
        {
            "bitcoin": initial_bitcoins,
            "cash": 0,
            "exit_amount": initial_bitcoins * last_closing_amount,
            "sell_day": "NA",
            "load_day": "NA",
        }
    ]

    # Only sell
    only_sell_index = 0
    for i in range(1, number_of_days):
        if data["open"][i] > data["open"][only_sell_index]:
            only_sell_index = i
    only_sell_entry = data.loc[only_sell_index]
    strategies.append(
        {
            "bitcoin": 0,
            "cash": initial_bitcoins * only_sell_entry["open"],
            "exit_amount": initial_bitcoins * only_sell_entry["open"],
            "sell_day": str(data["date"][only_sell_index].date()),
            "load_day": "NA",
        }
    )

    # Sell and load
    sell_index = 0
    load_index = 0
    max_index = 0
    max_bitcoins = initial_bitcoins
    for i in range(1, number_of_days):
        entry = data.loc[i]
        sell_cash = initial_bitcoins * data["open"][max_index]
        load_bitcoins = sell_cash / entry["open"]
        if load_bitcoins > max_bitcoins:
            max_bitcoins = load_bitcoins
            sell_index = max_index
            load_index = i
        if entry["open"] > data["open"][max_index]:
            max_index = i
    strategies.append(
        {
            "bitcoin": max_bitcoins,
            "cash": 0,
            "exit_amount": max_bitcoins * last_closing_amount,
            "sell_day": str(data["date"][sell_index].date()),
            "load_day": str(data["date"][load_index].date()),
        }
    )

    # Evaluate for best strategy
    best_strategy = strategies[0]
    for i in range(len(strategies)):
        if strategies[i]["exit_amount"] > best_strategy["exit_amount"]:
            best_strategy = strategies[i]

    # Calculate average closing price
    total_avg_closing_price = data["average"].sum()
    average_closing_price = total_avg_closing_price / number_of_days

    return {
        "profit_loss": best_strategy["exit_amount"] - investment_amount,
        "total_investment_amount": investment_amount,
        "total_exit_amount": best_strategy["exit_amount"],
        "average_closing_price": average_closing_price,
        "sell_day": best_strategy["sell_day"],
        "load_day": best_strategy["load_day"],
        "final_bitcoins": best_strategy["bitcoin"],
    }

In [ ]:
# Preprocess for swing strategy
df = pd.DataFrame(predictions)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(by="date")
df.set_index("date", inplace=True)
dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
df.dropna(inplace=True)
df = df.reindex(dates, method="ffill")
df.reset_index(inplace=True)
df.rename(columns={"index": "date"}, inplace=True)
df

In [ ]:
initial_investment = 100000
result = swing_trading_strategy(df, initial_investment)
result